In [3]:
import asyncio

from pyinjective.async_client import AsyncClient
from pyinjective.client.model.pagination import PaginationOption
from pyinjective.core.network import Network
import pandas as pd 

async def sub_order(account_address) -> None:
    network = Network.local()
    client = AsyncClient(network)
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]

async def order_list(subaccount_id,market_id) -> None:
    # select network: local, testnet, mainnet
    network = Network.local()
    client = AsyncClient(network)
    skip = 0
    limit = 100
    pagination = PaginationOption(skip=skip, limit=limit)
    orders = await client.fetch_subaccount_orders_list(
        subaccount_id=subaccount_id, market_id=market_id, pagination=pagination
    )
    return orders['orders']

async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    #print(df)
    df = df[["orderSide","subaccountId","price","quantity"]]
    df["price"] = df["price"].astype(float)/1e6
    df["quantity"] = df["quantity"].astype(float)
    df["quantity_in_$"] = df["price"] * df["quantity"]
    df = df[["orderSide","subaccountId","price","quantity_in_$"]]
    return df



In [4]:
from functions import marketId_to_ticker

ticker = "SAGA/USDT"
address = "inj14k7wgey85pt226ugr9sq4h2ya774n82hzy26nf"

market_id = await marketId_to_ticker(ticker)
print(market_id)
await subfinder(account_address=address,market_id=market_id[0])

['0xa508cb32923323679f29a032c70342c147c17d0145625922b0ef22e955c844c0']


KeyError: "None of [Index(['orderSide', 'subaccountId', 'price', 'quantity'], dtype='object')] are in the [columns]"